# Setup

In [ ]:
!pip install scipy==1.6.3
!pip install scikit_optimize==0.8.1
!pip install scikit_learn==0.24.2
! git clone https://github.com/zykhoo/predicting_hamiltonian_dynamics.git

     |████████████████████████████████| 27.4 MB 71.1 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.
     |████████████████████████████████| 101 kB 9.4 MB/s 
     |████████████████████████████████| 22.3 MB 1.4 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.4 requires scikit-learn>=1.0.0, but 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Run all

In [ ]:

"""# Experiment setup"""

import numpy as np

experiment,sys,dim = "NN","logarithm",2

H = lambda x: np.log(1+(x[0]*x[1])**2)
f1 = lambda x: 2*x[1]*x[0]**2/(1+(x[0]*x[1])**2) # dH/dy
f2 = lambda x: -2*x[0]*x[1]**2/(1+(x[0]*x[1])**2) # -dH/dx
spacedim = [(0.,4.),(0.,4.)]
fvector = lambda x: np.asarray([f1(x), f2(x)])


h= 0.1
x0, H0 = 0.,0.
initialcon = [64, 128, 256, 512, 1024]
LR = 0.001
diagdist = np.sum(np.square(np.asarray([spacedim[0][0], spacedim[0][1]]), np.asarray([spacedim[1][0], spacedim[1][1]])))
epsilon = 0.1

"""# Test dataset creation"""

from predicting_hamiltonian_dynamics import groundtruth_2dim
from tqdm import tqdm
import time 

evaluation_length_long = 50
evaluation_length_one = 1

xxlong,yylong = np.linspace(spacedim[0][0], spacedim[0][1], 5), np.linspace(spacedim[1][0], spacedim[1][1], 5)
xlong,ylong = np.meshgrid(xxlong,yylong)
xxshort,yyshort = np.linspace(spacedim[0][0], spacedim[0][1], 20), np.linspace(spacedim[1][0], spacedim[1][1], 20)
xshort,yshort = np.meshgrid(xxshort,yyshort)

trajectories_groundtruth_start_long = np.expand_dims(groundtruth_2dim.classicTrajectory(np.asarray([[0.4],[0.]]),f1,f2,h = 0.1,N=evaluation_length_long,n_h = 1),0)
for i in tqdm(np.expand_dims(np.c_[np.ravel(xlong),np.ravel(ylong)],2)):
  results_200 = np.expand_dims(groundtruth_2dim.classicTrajectory(i,f1,f2,h = 0.1,N=evaluation_length_long,n_h = 1),0)
  trajectories_groundtruth_start_long = np.vstack((trajectories_groundtruth_start_long, results_200))
trajectories_groundtruth_start_long.shape # output no., start and final, p and q, full traj

trajectories_groundtruth_start_short = np.expand_dims(groundtruth_2dim.classicTrajectory(np.asarray([[0.4],[0.]]),f1,f2,h = 0.1,N=evaluation_length_one,n_h = 1),0)
for i in tqdm(np.expand_dims(np.c_[np.ravel(xshort),np.ravel(yshort)],2)):
  results_200 = np.expand_dims(groundtruth_2dim.classicTrajectory(i,f1,f2,h = 0.1,N=evaluation_length_one,n_h = 1),0)
  trajectories_groundtruth_start_short = np.vstack((trajectories_groundtruth_start_short, results_200))
trajectories_groundtruth_start_short.shape # output no., start and final, p and q, full traj

within_array = groundtruth_2dim.get_within_array(trajectories_groundtruth_start_long, spacedim)

100%|██████████| 400/400 [00:00<00:00, 2370.67it/s]


In [ ]:
path = '/content/drive/MyDrive/SSI/Baseline v2/synthetic systems (upload)/'

In [ ]:
from predicting_hamiltonian_dynamics.models import NN_2dim
from predicting_hamiltonian_dynamics.models import GP_2dim
from predicting_hamiltonian_dynamics.models import PINN_2dim
from predicting_hamiltonian_dynamics.models import PIGP_2dim


for i in range(20):
  seed = i
  np.random.seed(seed=seed)
  for ini in initialcon: 

    start, final = groundtruth_2dim.CreateTrainingDataTrajClassicIntRandom(1,ini,spacedim,h,f1,f2,seed = seed,n_h = 1)

    delta = start.copy()
    delta[0,:] = f1(start)
    delta[1,:] = f2(start)

    """# NN"""

    from predicting_hamiltonian_dynamics.models import NN_2dim
    import torch

    if torch.cuda.is_available():
      device=torch.device('cuda')
    else:
      device=torch.device('cpu')
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


    wholemat, evalmat = NN_2dim.data_preprocessing(start, delta, device)

    import torch.optim as optim
    import time 

    net = NN_2dim.Net(dim,16,dim)
    starttime = time.time() 
    net = NN_2dim.train(net, wholemat, evalmat, optimizer=optim.Adam(net.parameters(), lr=LR), batchsize=10, iter=1600, )
    traintime = time.time()-starttime


    from tqdm import tqdm
    from predicting_hamiltonian_dynamics.models import NN_2dim
    from predicting_hamiltonian_dynamics import metrics


    MSE_long, time_long, MSE_long_naive, time_long_naive, MSE_within, time_within, MSE_within_naive, time_within_naive, MSE_onestep, time_onestep, MSE_vectorfield, time_vectorfield, _, _ = NN_2dim.compute_metrics_NN(net, h, diagdist, xshort, yshort, xlong, ylong, evaluation_length_long, within_array, trajectories_groundtruth_start_long, trajectories_groundtruth_start_short, fvector)

    file_object = open(path + 'Logarithmic_random.txt', 'a')
    file_object.write('NN, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s \n' %(ini, seed, traintime, MSE_long, time_long, MSE_long_naive, time_long_naive, MSE_within, time_within, MSE_within_naive, time_within_naive, MSE_onestep, time_onestep, MSE_vectorfield, time_vectorfield))
    file_object.close()


    """# GP"""

    from sklearn.gaussian_process import GaussianProcessRegressor
    from sklearn.gaussian_process.kernels import RBF
    from sklearn import preprocessing

    output = delta
    scaler = preprocessing.MinMaxScaler()
    input = scaler.fit_transform(start.transpose())
    kernel = 1 * RBF(length_scale=2., length_scale_bounds="fixed")
    gaussian_process = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=0, optimizer = None, random_state = seed)
    starttime = time.time() 
    gaussian_process.fit(input, output.transpose())
    traintime = time.time()-starttime
    gaussian_process.predict(scaler.transform([[0.7,0.8]]))

    MSE_long, time_long, MSE_long_naive, time_long_naive, MSE_within, time_within, MSE_within_naive, time_within_naive, MSE_onestep, time_onestep, MSE_vectorfield, time_vectorfield, _, _ = GP_2dim.compute_metrics_GP(gaussian_process, scaler, h, diagdist, xshort, yshort, xlong, ylong, evaluation_length_long, within_array, trajectories_groundtruth_start_long, trajectories_groundtruth_start_short, fvector)

    file_object = open(path + 'Logarithmic_random.txt', 'a')
    file_object.write('GP, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s \n' %(ini, seed, traintime, MSE_long, time_long, MSE_long_naive, time_long_naive, MSE_within, time_within, MSE_within_naive, time_within_naive, MSE_onestep, time_onestep, MSE_vectorfield, time_vectorfield))
    file_object.close()


    """# PINN"""

    from predicting_hamiltonian_dynamics.models import PINN_2dim
    import torch

    if torch.cuda.is_available():
      device=torch.device('cuda')
    else:
      device=torch.device('cpu')
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    wholemat, evalmat = PINN_2dim.data_preprocessing(start, delta,device)
    net=PINN_2dim.Net(dim,16,1)
    starttime = time.time()
    net,avg_vallosses,avg_lossli,avg_f1li,avg_f2li,avg_f3li,avg_f4li=PINN_2dim.train(net,bs=10,num_epoch=5000,initial_conditions=initialcon,device=device,
                                                                                    wholemat=wholemat,evalmat=evalmat,x0=x0,H0=H0,dim=dim,LR=LR,patience=500,c1=1,c2=1,c3=1,c4=1)
    traintime = time.time()-starttime

    MSE_long, time_long, MSE_long_naive, time_long_naive, MSE_within, time_within, MSE_within_naive, time_within_naive, MSE_onestep, time_onestep, MSE_vectorfield, time_vectorfield, _, _ = PINN_2dim.compute_metrics_PINN(net, device, h, diagdist, xshort, yshort, xlong, ylong, evaluation_length_long, within_array, trajectories_groundtruth_start_long, trajectories_groundtruth_start_short, fvector)
    
    file_object = open(path + 'Logarithmic_random.txt', 'a')
    file_object.write('PINN, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s \n' %(ini, seed, traintime, MSE_long, time_long, MSE_long_naive, time_long_naive, MSE_within, time_within, MSE_within_naive, time_within_naive, MSE_onestep, time_onestep, MSE_vectorfield, time_vectorfield))
    file_object.close()

    """# PIGP"""

    from predicting_hamiltonian_dynamics.models import PIGP_2dim
    GP = PIGP_2dim.BertalanGP()
    starttime = time.time()
    GP.train(start,delta,h)
    traintime = time.time()-starttime

    MSE_long, time_long, MSE_long_naive, time_long_naive, MSE_within, time_within, MSE_within_naive, time_within_naive, MSE_onestep, time_onestep, MSE_vectorfield, time_vectorfield, _, _ = PIGP_2dim.compute_metrics_PIGP(GP, h, diagdist, xshort, yshort, xlong, ylong, evaluation_length_long, within_array, trajectories_groundtruth_start_long, trajectories_groundtruth_start_short, fvector)

    file_object = open(path + 'Logarithmic_random.txt', 'a')
    file_object.write('PIGP, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s \n' %(ini, seed, traintime, MSE_long, time_long, MSE_long_naive, time_long_naive, MSE_within, time_within, MSE_within_naive, time_within_naive, MSE_onestep, time_onestep, MSE_vectorfield, time_vectorfield))
    file_object.close()


training loss tensor(1.8911, dtype=torch.float64, grad_fn=<MeanBackward0>)
validation loss tensor(1.2700, dtype=torch.float64, grad_fn=<MeanBackward0>)
training loss tensor(0.3699, dtype=torch.float64, grad_fn=<MeanBackward0>)
validation loss tensor(0.1354, dtype=torch.float64, grad_fn=<MeanBackward0>)
training loss tensor(0.1785, dtype=torch.float64, grad_fn=<MeanBackward0>)
validation loss tensor(0.0815, dtype=torch.float64, grad_fn=<MeanBackward0>)
training loss tensor(0.1262, dtype=torch.float64, grad_fn=<MeanBackward0>)
validation loss tensor(0.0554, dtype=torch.float64, grad_fn=<MeanBackward0>)
training loss tensor(0.1005, dtype=torch.float64, grad_fn=<MeanBackward0>)
validation loss tensor(0.0451, dtype=torch.float64, grad_fn=<MeanBackward0>)
training loss tensor(0.0860, dtype=torch.float64, grad_fn=<MeanBackward0>)
validation loss tensor(0.0398, dtype=torch.float64, grad_fn=<MeanBackward0>)
training loss tensor(0.0720, dtype=torch.float64, grad_fn=<MeanBackward0>)
validation lo

  0%|          | 0/50 [00:00<?, ?it/s]/content/predicting_hamiltonian_dynamics/models/NN_2dim.py:25: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  fstage = lambda stg: h * torch.squeeze(net(torch.tensor([q+stg,p]).float().transpose(1,0)),0).detach().numpy().transpose()[:dim]
100%|██████████| 50/50 [00:00<00:00, 218.12it/s]

100%|██████████| 50/50 [00:00<00:00, 2816.75it/s]

100%|██████████| 50/50 [00:00<00:00, 240.97it/s]

100%|██████████| 50/50 [00:00<00:00, 180.22it/s]

100%|██████████| 50/50 [00:00<00:00, 2565.76it/s]

100%|██████████| 50/50 [00:00<00:00, 216.00it/s]

100%|██████████| 50/50 [00:00<00:00, 384.72it/s]

100%|██████████| 50/50 [00:00<00:00, 1397.15it/s]

100%|██████████| 50/50 [00:00<00:00, 329.75it/s]

100%|██████████| 50/50 [00:00<00:00, 337.21it/s]

100%|█

 
epoch= 0  time= 0.02336573600769043  loss= 3.0442335506280265  val_loss= 5.599068999290466  f1= 1.3669979620551447  f2= 1.6693443382545177  f3= 0.00724839970159034  f4= 0.0006429548691169074 percent lr= 0.001
 
epoch= 10  time= 0.16997241973876953  loss= 2.9631287852923074  val_loss= 4.791617155075073  f1= 1.2719502797639162  f2= 1.678422678575502  f3= 0.004659977741539478  f4= 0.00809586130139872 percent lr= 0.001
 
epoch= 20  time= 0.3082001209259033  loss= 3.3442936738332114  val_loss= 3.438642740249634  f1= 2.277347748371584  f2= 1.03358761762573  f3= 0.00759966392070055  f4= 0.02575856019300135 percent lr= 0.001
 
epoch= 30  time= 0.4498097896575928  loss= 1.3290923088788986  val_loss= 1.7350475490093231  f1= 0.673903356659023  f2= 0.6062258685825196  f3= 0.00590616837143898  f4= 0.0430569218330897 percent lr= 0.001
 
epoch= 40  time= 0.5932481288909912  loss= 0.65738778313001  val_loss= 0.438037633895874  f1= 0.34247779644898957  f2= 0.2614389014218175  f3= 0.000835343186433116

100%|██████████| 400/400 [00:01<00:00, 281.73it/s]


Start training with for 64 data points
Start computation of covariance matrix.
Covariance matrix of shape (64, 64)computed.
Start Cholesky decomposition of (64, 64) Matrix
Cholesky decomposition completed.
Create LHS of linear system for H at test points.
Creation of linear system completed.
Solve least square problem of dimension (129, 64)


...

100%|██████████| 50/50 [00:00<00:00, 211.43it/s]

100%|██████████| 50/50 [00:00<00:00, 2604.90it/s]

100%|██████████| 50/50 [00:00<00:00, 192.04it/s]

100%|██████████| 50/50 [00:00<00:00, 317.78it/s]

100%|██████████| 50/50 [00:00<00:00, 2790.62it/s]

100%|██████████| 50/50 [00:00<00:00, 309.68it/s]

100%|██████████| 50/50 [00:00<00:00, 319.21it/s]

100%|██████████| 50/50 [00:00<00:00, 3264.30it/s]

100%|██████████| 50/50 [00:00<00:00, 308.85it/s]

100%|██████████| 50/50 [00:00<00:00, 371.50it/s]

100%|██████████| 50/50 [00:00<00:00, 3092.83it/s]

100%|██████████| 50/50 [00:00<00:00, 347.41it/s]

100%|██████████| 50/50 [00:00<00:00, 534.58it/s]

100%|██████████| 50/50 [00:00<00:00, 2842.29it/s]

100%|██████████| 50/50 [00:00<00:00, 538.60it/s]

100%|██████████| 50/50 [00:00<00:00, 162.14it/s]

100%|██████████| 50/50 [00:00<00:00, 3196.05it/s]

100%|██████████| 50/50 [00:00<00:00, 152.07it/s]

100%|██████████| 50/50 [00:00<00:00, 257.35it/s]

100%|██████████| 50/50 [00:00<00:00, 2966.98

 
epoch= 0  time= 0.22519207000732422  loss= 2.0537608956418385  val_loss= 1.5892348378896712  f1= 0.8426376870515887  f2= 1.1300142168208533  f3= 0.017288192496804064  f4= 0.0638208104645246 percent lr= 0.001
 
epoch= 10  time= 2.582990884780884  loss= 0.22950842167909552  val_loss= 0.19114360641688108  f1= 0.09355392400919856  f2= 0.12488225716541984  f3= 0.00010919314180161582  f4= 0.010963047971287058 percent lr= 0.001
 
epoch= 20  time= 4.935006380081177  loss= 0.12295785499736667  val_loss= 0.09089192752726376  f1= 0.051467418915248725  f2= 0.061349234826597515  f3= 4.282741672066748e-05  f4= 0.010098373559841773 percent lr= 0.001
 
epoch= 30  time= 7.282520532608032  loss= 0.09399787253677482  val_loss= 0.06691410331986844  f1= 0.04047331673507118  f2= 0.04535040068744476  f3= 5.815636173661518e-05  f4= 0.008115999114213967 percent lr= 0.001
 
epoch= 40  time= 9.602285146713257  loss= 0.07238597943299882  val_loss= 0.04962636269629002  f1= 0.03289375570921594  f2= 0.033884104186

100%|██████████| 400/400 [00:01<00:00, 261.38it/s]


Start training with for 1024 data points
Start computation of covariance matrix.
Covariance matrix of shape (1024, 1024)computed.
Start Cholesky decomposition of (1024, 1024) Matrix
Cholesky decomposition completed.
Create LHS of linear system for H at test points.
Creation of linear system completed.
Solve least square problem of dimension (2049, 1024)


100%|██████████| 400/400 [08:20<00:00,  1.25s/it]
